# 第二题：实现一个高斯朴素贝叶斯分类器

实验内容：
1. 实现高斯朴素贝叶斯分类器
2. 计算模型的查准率，查全率，F1值

# 1. 导入数据集

In [1]:
import numpy as np

In [2]:
spambase = np.loadtxt('data/spambase/spambase.data', delimiter = ",")
spamx = spambase[:, :57]
spamy = spambase[:, 57]

# 2. 划分数据集

In [3]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(spamx, spamy, test_size = 0.4, random_state = 32)

In [4]:
trainX.shape, trainY.shape, testX.shape, testY.shape

((2760, 57), (2760,), (1841, 57), (1841,))

# 3. 实现高斯朴素贝叶斯

朴素贝叶斯的实现非常简单，但是首先需要大家掌握几个技巧的使用

### 1. python dict

python的字典，给定字典a，使用`a[key] = value`，将 `{key: value}` 键值对添加进a中

In [5]:
test_dict = dict()
# 将 {'a': 1} 存入test_dict
# YOUR CODE HERE
test_dict = {'a': 1}

In [6]:
print(test_dict['a']) # 1

1


### 2. np.mean

求均值，使用 `axis = 0` 这个参数对每列取均值，`axis = 1` 对每行取均值，使用 `keepdims = True` 使结果保持之前的维数。

In [7]:
test_matrix = spamx[spamy == 1, :]
print(test_matrix.shape)

(1813, 57)


In [8]:
# 求 test_matrix 每列的均值，存入test_mean中
# YOUR CODE HERE
test_mean = np.mean(test_matrix, axis=0)

In [9]:
print(test_mean.sum()) # 595.062044126
print(test_mean.shape) # (1, 57)

595.0620441257583
(57,)


### 3. np.var

求方差，使用 `axis = 0` 这个参数对每列取方差，`axis = 1` 对每行取方差，使用 `keepdims = True` 使结果保持之前的维数。

In [10]:
# 求 test_matrix 每列的方差，存入test_var中
# YOUR CODE HERE
test_var = np.var(test_matrix, axis=0)

In [11]:
# 测试样例
print(test_var.sum()) # 772407.506004
print(test_var.shape) # (1, 57)

772407.5060043654
(57,)


### 4. 将test_mean和test_var存入字典test_dict中

将`{'mean': test_mean}` 和 `{'var': test_var}` 存入`test_dict`中

In [12]:
# YOUR CODE HERE
test_dict['mean'] = test_mean
test_dict['var'] = test_var

In [13]:
# 测试样例
print(test_dict.keys())  # dict_keys(['a', 'mean', 'var'])

dict_keys(['a', 'mean', 'var'])


### 5. numpy的索引

我们在预测的时候，需要使用numpy索引的一个小技巧

给定一个列表，里面有3个字符串`'a', 'b', 'c'`，分别表示三个类别，给定一个`np.ndarray([1, 2, 0, 1, 0])`，我们可以执行以下代码观察numpy强大的索引功能

In [14]:
labels = ['a', 'b', 'c']
np_labels = np.array(labels)
print(np_labels)
index = np.array([1, 2, 0, 1, 0])
print(index)
results = np_labels[index]
print(results)
print(type(results))

['a' 'b' 'c']
[1 2 0 1 0]
['b' 'c' 'a' 'b' 'a']
<class 'numpy.ndarray'>


可以看到，我们可以使用 `np.ndarray` 一次检索多个值，返回值会以 `np.ndarray` 的形式返回

### 6. np.argmax

这个是求最大值的下标用的，`axis`参数用来控制是每行还是每列

In [15]:
test_array = np.array([[0.9, 0.1],[0.4, 0.6],[0.1, 0.9]])
print(test_array)

[[0.9 0.1]
 [0.4 0.6]
 [0.1 0.9]]


In [16]:
np.argmax(test_array, axis = 0)

array([0, 2], dtype=int64)

可以看到，使用`axis = 0`，就是返回每列最大值的下标，分别是0和2。

In [17]:
# 求每行最大值的下标
# YOUR CODE HERE
test_argmax = np.argmax(test_matrix, axis=1)

In [18]:
print(test_argmax) # [0 1 1]

[56 56 56 ... 56 56 56]


**接下来我们开始实现高斯朴素贝叶斯**，我们以类的形式实现这个高斯朴素贝叶斯。因为朴素贝叶斯是懒惰学习，所以这个模型只有在预测的时候，会进行大量的运算。

In [19]:
class myGaussianNB:
    '''
    处理连续特征的高斯朴素贝叶斯
    '''
    def __init__(self):
        '''
        初始化四个字典
        self.label_mapping     类标记 与 下标(int)
        self.probability_of_y  类标记 与 先验概率(float)
        self.mean              类标记 与 均值(np.ndarray)
        self.var               类标记 与 方差(np.ndarray)
        '''
        self.label_mapping = dict()
        self.probability_of_y = dict()
        self.mean = dict()
        self.var = dict()
        
    def _clear(self):
        '''
        为了防止一个实例反复的调用fit方法，我们需要每次调用fit前，将之前学习到的参数删除掉
        '''
        self.label_mapping.clear()
        self.probability_of_y.clear()
        self.mean.clear()
        self.var.clear()
    
    def fit(self, trainX, trainY):
        '''
        这里，我们要根据trainY内的类标记，针对每类，计算这类的先验概率，以及这类训练样本每个特征的均值和方差
        '''
        self._clear()
        labels = np.unique(trainY)
        self.label_mapping = {label: index for index, label in enumerate(labels)}

        for label in labels:
            x = trainX[trainY == label, :]
            # 计算先验概率，用 x 的样本个数除以训练样本总个数，存储到 self.probability_of_y 中，键为 label，值为先验概率
            # YOUR CODE HERE
            self.probability_of_y[label] = x.shape[0] / trainX.shape[0]
            # 对 x 的每列求均值，使用 keepdims = True 保持维度，存储到 self.mean 中，键为 label，值为每列的均值组成的一个二维 np.ndarray
            # YOUR CODE HERE
            self.mean[label] = np.mean(x, axis=0, keepdims=True)
            assert self.mean[label].shape == (1, trainX.shape[1])
            # 对 x 的每列求方差，使用 keepdims = True 保持维度，存储到 self.var 中，键为 label，值为每列的方差组成的一个二维 np.ndarray
            # YOUR CODE HERE
            self.var[label] = np.var(x, axis=0, keepdims=True)
            assert self.var[label].shape == (1, trainX.shape[1])
            self.var[label] += 1e-9 * np.var(trainX, axis = 0).max()
        
    def predict(self, testX):
        results = np.empty((testX.shape[0], len(self.probability_of_y)))
        labels = [0] * len(self.probability_of_y)
        for label, index in self.label_mapping.items():
            py = self.probability_of_y[label]
            # 使用变换后的公式，计算所有特征的条件概率之和，存为sum_of_conditional_probability
            # YOUR CODE HERE
            sum_of_conditional_probability = -0.5 * np.sum(np.log(2. * np.pi * self.var[label]) + (testX - self.mean[label]) ** 2 / self.var[label], axis=1)
            assert sum_of_conditional_probability.shape == (len(testX), )
            # 使用变换后的公式，将 条件概率 与 log先验概率 相加，存为result，维度应该是 (测试样本数, )
            # YOUR CODE HERE
            result = sum_of_conditional_probability + np.log(py)
            assert result.shape == (len(testX), )
            results[:, index] = result
            labels[index] = label
        
        np_labels = np.array(labels)
        # 循环结束后，就计算出了给定测试样本，当前样本属于这类的概率的近似值，存放在了results中，每行对应一个样本，每列对应一个特征
        # 我们要求每行的最大值对应的下标，也就是求每个样本，概率值最大的那个下标是什么，结果存入max_prob_index中
        # YOUR CODE HERE
        max_prob_index = np.argmax(results, axis=1)
        assert max_prob_index.shape == (len(testX), )
        # 现在得到了每个样本最大概率对应的下标，我们需要把这个下标变成 np_labels 中的标记
        # 使用上面小技巧中的第五点求解
        # YOUR CODE HERE
        prediction = np_labels[max_prob_index]
        assert prediction.shape == (len(testX), )
        return prediction

In [20]:
from sklearn.metrics import accuracy_score
model = myGaussianNB()
model.fit(trainX, trainY)
accuracy_score(testY, model.predict(testX))  # 0.812

0.8126018468223791

# 4. 计算其他的指标

###### 双击此处填写

查准率|查全率|F1
-|-|-
0.682951|0.962079| 0.798834

In [21]:
# YOUR CODE HERE
from sklearn.metrics import precision_score, recall_score, f1_score
predY = model.predict(testX)
precision = precision_score(testY, predY)
recall = recall_score(testY, predY)
f1 = f1_score(testY, predY)
print("precision: ", format(precision,'.6f'))
print("recall: ", format(recall,'.6f'))
print("F1: ", format(f1,'.6f'))

precision:  0.682951
recall:  0.962079
F1:  0.798834
